# Testing

```source: this key separates the various keys found in the table in Sources. Here's the set of sources with their corresponding value name:```
```
'https://aipulse.org'
'ebook'
'https://qualiacomputing.com'
'alignment forum'
'lesswrong'
'manual'
'arxiv'
'https://deepmindsafetyresearch.medium.com/'
'waitbutwhy.com'
'GitHub'
'https://aiimpacts.org'
'arbital.com'
'carado.moe'
'nonarxiv_papers'
'https://vkrakovna.wordpress.com'
'https://jsteinhardt.wordpress.com'
'audio-transcripts'
'https://intelligence.org'
'youtube'
'reports'
'https://aisafety.camp'
'curriculum'
'https://www.yudkowsky.net'
'distill'
```

## Imports

In [22]:
import jsonlines
import numpy as np
from typing import List, Dict, Tuple

## Constants

In [19]:
# None

## Helpers

In [20]:
# None

## Dataset Class

In [ ]:
class Dataset:
    def __init__(self, path: str, sources: List[str] = None, max_data_length: int = 1e6, find_embeddings: bool = False, len_embeddings: int = 1536):
        self.data = {}
        self.sources = sources
        self.max_data_length = max_data_length
        self.len_embeddings = len_embeddings
        
        self.data = self.load(path, sources, max_data_length)
        self.data = self.process()
        
        self.num_embeddings = len(self.data)
        self.embeddings = np.zeros((self.num_embeddings, self.len_embeddings))
        if find_embeddings:
            self.embeddings = self.get_embeddings()
        
    def load(self, path: str, sources: List[str], max_data_length: int):
        i = 0
        with jsonlines.open(path, "r") as reader:
            for entry in reader:
                try:
                    if entry['source'] in sources:
                        self.data[i] = entry['text']
                        i += 1
                except KeyError:
                    pass
        return self.data    
                
    def process(self):
        raise NotImplementedError
    
    def get_embeddings(self):
        raise NotImplementedError

In [ ]:
alignment_dataset = r"C:\Users\Henri\Documents\GitHub\AlignmentSearch\data\alignment_texts.jsonl"

worthwhile_sources = [
    # 'alignment forum',
    # 'lesswrong',
    'manual',
    # 'GitHub',
    'https://jsteinhardt.wordpress.com',
    'https://intelligence.org',
    'https://www.yudkowsky.net',
    'distill'
]


In [15]:
dataset = Dataset(path=alignment_dataset, sources=worthwhile_sources, max_data_length=1e4, get_embeddings=False)

{'title': 'Webinar with Congressman Ro Khanna: Challenges in IT Law and Governance', 'title_detail': {'type': 'text/plain', 'language': None, 'base': 'https://aipulse.org/feed/', 'value': 'Webinar with Congressman Ro Khanna: Challenges in IT Law and Governance'}, 'links': [{'rel': 'alternate', 'type': 'text/html', 'href': 'https://aipulse.org/webinar-with-congressman-ro-khanna-challenges-in-it-law-and-governance/'}], 'link': 'https://aipulse.org/webinar-with-congressman-ro-khanna-challenges-in-it-law-and-governance/', 'authors': [{'name': 'Laura Elbaum'}], 'author': 'Laura Elbaum', 'author_detail': {'name': 'Laura Elbaum'}, 'published': 'Mon, 22 Feb 2021 16:40:08 +0000', 'published_parsed': [2021, 2, 22, 16, 40, 8, 0, 53, 0], 'tags': [{'term': 'Webinars', 'scheme': None, 'label': None}], 'id': 'https://aipulse.org/?p=712', 'guidislink': False, 'summary': '<p>On Friday, February 19, the AI Pulse project hosted a web conversation on current issues in IT Governance with Congressman Ro Kha

In [14]:
print(dictionary[0])

Girl Intercorrupted

This is a 4-of-13 chapter sample of “A Girl Corrupted by the Internet is the Summoned Hero?!” The remainder is available at Gumroad and Amazon .
Table of Contents
Prologue: A virgin maiden is already corrupted?!The chance of success is—?!The key to power is—!?The rebellion has already lost?!I’m going to be sacrificed?!Is this my story’s shocking twist?!The true key to power is—?!Do even I dare?!Am I going to wimp out?!Do you really think you can?!The meaning of probability is—You did it because—?!The final bargain!
©2016 by Eliezer Yudkowsky.
Foreword
This is my attempt at translating a light novel from Japan, only the original source material doesn’t exist.
The light novel is a Japanese custom which aims at easy reading. I think of it as an art form in which only the story’s bones remain.
If you want to read a translation of a Japanese light novel, I liked “Evil God Average” (Jashin Average) as translated by the Fifth Holy Sheeprabbit. That might help you to appre